In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os
from datetime import datetime, timedelta
import uuid
import random

In [2]:
# Set the environment variable to use the local Firestore emulator
os.environ["FIRESTORE_EMULATOR_HOST"] = "host.docker.internal:8081"

In [3]:
cred = credentials.Certificate("./_serviceAccount.json")
firebase_admin.initialize_app(cred)

In [4]:
db = firestore.client()

In [8]:
# Execute the query and get the results
wht_collection = db.collection('whtCodes')

In [12]:
# Execute the query and get the results
doc_ref = wht_collection.document('SG')
doc_ref.set({
    "whtCodes": [{
        "code": "1A",
        "isImportant": True,
        "name": "WC110",
        "perc": 5,
        "sourceId": "1"
    }, {
        "code": "2A",
        "name": "WC117",
        "perc": 2,
        "sourceId": "2"
    }, {
        "code": "WHT15",
        "isImportant": True,
        "name": "WHT15%",
        "perc": 15,
        "sourceId": "3"
    }, {
        "code": "WHT10",
        "isImportant": True,
        "name": "WHT10%",
        "perc": 10,
        "sourceId": "4"
    }]
})

update_time {
  seconds: 1725797609
  nanos: 192093000
}

In [20]:
def generate_sample_bill(bill_number):
    """Generate a sample bill with the given bill number."""
    return {
        "action": "create",
        "billId": str(uuid.uuid4()),
        "vendorId": "wFW2BNhegaFQBM0TTboR",
        "vendorName": "Test Vendor From NS Intg 4",
        "vendorAccountNumber": f"ACC-{random.randint(10000, 99999)}",
        "billStatus": random.choice(["draft", "scheduled", "paid"]),
        "billNumber": bill_number,
        "lineItems": [
            {
                "name": f"Item {i}",
                "quantity": random.randint(1, 10),
                "unitPrice": round(random.uniform(10, 1000), 2),
                "account": "Bank Fees",
                "accountId": "fEhhLF52zpNMznntWhi1",
                "type": "expense",
                "wht": {
                    "referenceId": f"WHT-{random.randint(1000, 9999)}",
                    "code": "1A",
                    "displayName": "WC110",
                    # "perc": random.randint(1, 10)
                    "perc": 11
                }
            } for i in range(random.randint(1, 5))
        ],
        "total": 0,  # We'll calculate this based on line items
        "subTotal": 0,  # We'll calculate this based on line items
        "paymentAmount": 0,  # We'll calculate this based on total
        "date": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
        "dueDate": (datetime.now() + timedelta(days=random.randint(1, 30))).strftime("%Y-%m-%d"),
        "currency": 'SGD',
        "currencyRate": round(random.uniform(0.8, 1.2), 4),
        "isVendorSynced": True,
        "peakfloId": f"PF-{random.randint(10000, 99999)}",
        "billCreatedAt": datetime.now().isoformat(),
        "type": "BILL",
        "status": "ready",
        "customerId": "demo1Company",
        "updatedAt": firestore.SERVER_TIMESTAMP
    }

def calculate_bill_totals(bill):
    """Calculate totals for the bill based on line items."""
    sub_total = 0
    total_wht = 0
    
    for item in bill["lineItems"]:
        quantity = item["quantity"]
        unit_price = item["unitPrice"]
        item_amount = round(quantity * unit_price, 2)
        item["amount"] = item_amount
        sub_total += item_amount
        
        if "wht" in item:
            wht_perc = item["wht"]["perc"]
            wht_amount = round(item_amount * (wht_perc / 100), 2)
            item["wht"]["amount"] = wht_amount
            total_wht += wht_amount
    
    bill["subTotal"] = round(sub_total, 2)
    bill["totalWHT"] = round(total_wht, 2)
    bill["total"] = round(sub_total - total_wht, 2)
    bill["paymentAmount"] = bill["total"]

def insert_bills_to_firestore(num_bills=10):
    """Insert the specified number of bills into Firestore."""
    collection_ref = db.collection("netsuite-bill-outgoing-updates")
    
    for i in range(1, num_bills + 1):
        bill_number = f"BILL-{datetime.now().strftime('%Y%m%d')}-{i:03d}"
        bill = generate_sample_bill(bill_number)
        calculate_bill_totals(bill)
        
        # add bill
        collection_ref.add(bill)
        
        print(f"Inserted bill with number: {bill_number}")

In [21]:
insert_bills_to_firestore(1)

Inserted bill with number: BILL-20240908-001
